In [1]:
library(readxl)
df <- read_excel("AGC_YPD30.xlsx")

In [ ]:
library(pracma)  # for trapz()


# Extract time
time <- df[[1]]

# Function to calculate doubling  time
calc_doubling_time <- function(time, values) {
  valid_idx <- which(!is.na(values) & values > 0)
  if (length(valid_idx) < 2) return(NA)
  
  log_values <- log(values[valid_idx])
  time_vals  <- time[valid_idx]
  
  # Fit log-linear model
  fit <- lm(log_values ~ time_vals)
  rate <- coef(fit)[2]
  
  if (rate == 0) return(NA)
  
  if (rate > 0) {
    # Positive growth → doubling time
    return(log(2) / rate)
  } else {
    # Negative growth → halving time (negative indicates decay)
    return(-log(2) / abs(rate))
  }
}

# Loop through columns and calculate metrics
results <- data.frame(Column = character(),
                      AUC = numeric(),
                      DoublingTime = numeric(),
                      stringsAsFactors = FALSE)

for (colname in names(df)[-1]) {
  values <- df[[colname]]

  if (is.numeric(values)) {
    # Replace NAs with 0 for AUC calculation
    auc <- trapz(time, ifelse(is.na(values), 0, values))
    dt  <- calc_doubling_time(time, values)
    
    results <- rbind(results, 
                     data.frame(Column = colname, AUC = auc, DoublingTime = dt))
  }
}

print(results)
write.csv(results, "GC_results_YPD30.csv", row.names = FALSE)

           Column      AUC DoublingTime
time_vals     110 14.58265     2.852197
time_vals1    111 16.01205     2.994947
time_vals2    112 17.72627     3.044804
time_vals3    AE2 14.09966     2.733417
time_vals4    BG2 17.88049     3.761936
time_vals5 CBS138 15.11871     2.137052
time_vals6     D1 17.65167     2.712811
